# Проект для кредитного отдела банка.

## Влияет ли семейное положение и наличие детей у клиента на возврат кредита в срок.

1. [**Подготовка данных к анализу.**](#100101)
2. [**Проблемы на первый взгляд.**](#100102)
3. [**Новые проблемы в данных.**](#100103)
4. [**Стемминг.**](#100104)
5. [**Задолженность у клиентов с разным количеством детей.**](#100105)
6. [**Задолженность клиентов в зависимости от их семейного статуса.**](#100106)
7. [**Группы по доходам.**](#100107)
8. [**Зависимость задолженности от - детей, семейного положения, доходов.**](#100108)
9. [**Выводы.**](#100109)

In [2]:
import pandas as pd
data=pd.read_csv('data.csv')

from nltk.stem import SnowballStemmer
from pymystem3 import Mystem
pd.options.mode.chained_assignment = None
pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500) 
pd.set_option('display.width', 1000)

### 1. Подготовка данных к анализу <a id='100101'></a>

In [3]:
#Посмотрим на таблицу и её параметры:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
#Посмотрим на названия колонок:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'], dtype='object')

In [5]:
#Распечатаем данные первых двадцати строк:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [6]:
#Распечатаем данные последних двадцати строк:
data.tail(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости


### 2. Обозначим проблемы на первый взгляд: <a id='100102'></a>
12 колонок и 21525 строк. Название столбца 'dob_days' лучше сменить 'age', для более правильного отражения содержимого.
* В колонке 'days_employed' есть отрицательные значения и пропуски (в колонке заполнена всего 19351 строка, кстати в 'total_income' тоже, между пропусками прослеживается прямая связь), а так же лучше привести эту колонку к целочисленному типу;
* В колонке 'education' есть проблемы с регистром;
* В колонке 'family_status' проблемы с регистром, лучше привести к единообразию;
* В колонке 'total_income' есть пропущенные значения и для удобства, ее лучше привести к целочисленному типу;
* Колонка purpose требует лемматизации.
*********************
### Проведем более подробное исследование:

In [7]:
#Проверим количество пропусков  методами **isnull()** и **sum()**
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Подтвердилась явная взаимосвязь пропусков двух столбцов 'days_employed' и 'total_income'.

* _Исследуем уникальные значения некоторых столбцов методами **value_counts()** и **unique()**:_

In [8]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [9]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

In [10]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [11]:
data['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [12]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [13]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [14]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [15]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [16]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [17]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

### 3.Выявились новые проблемы: <a id='100103'></a>
* в столбце 'children' - явно неправильные значения это 20 и -1, явно случайные опечатки при введении, исправим на 2 и 1
* в столбце 'dob-days' - значение 0, нужно выяснить почему и возможно удалить
* в столбце 'gender' - выявился третий пол XNA ;), хоть это сейчас и популярно, будем считать это ошибкой, а так же заменим тип столбца на строковый

### Начнём исправлять:

In [18]:
#Изменим название колонки 'dob_days' на 'age' методом **set_axis**:
new_names = (['children', 'days_employed', 'age', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'])

In [19]:
data.set_axis(new_names, axis='columns', inplace=True)

#проверим:
#data.columns

В некоторых ячейках 'age' неизвестен возраст клиентов (0) - найдем среднее и поставим средний возраст с помощью методов **median()** и **replace()**:

In [20]:
data['age'] = data['age'].astype(str)
data.loc[data['age'] == '0', 'age'] = data['age'].median()
data['age'] = data['age'].astype(int)

#проверим:
#data['age'].unique()

Изменим регистр в 'family-status' и 'education' применяя метод **str.lower()**:

In [21]:
data['family_status'] = data['family_status'].str.lower()
data['education'] = data['education'].str.lower()

#проверим:
#data['education'].unique()

**Заметно уменьшилось количество уникальных значений, но нужно проверить соответствие id, т.к. колонки связаны**

In [22]:
data.groupby('education')['education'].count()

education
высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64

In [23]:
data.groupby('education_id')['education_id'].count()

education_id
0     5260
1    15233
2      744
3      282
4        6
Name: education_id, dtype: int64

Из взгляда на таблицу понятно, какие значения каким ID соответствуют. Количество значений и Id тоже равны. Сделаем вывод, что при изменении столбца 'education' никаких ошибок не произошло.

In [24]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

здесь все по-прежнему, но в едином регистре, что удобно для дальнейших манипуляций.

* **'days_employed'** и **'total_income'** явно связаны, незаполненные строки в обоих столбцах соответствуют друг другу 
* (обратить внимание! что не заполнено около 10%) придется исключить их из анализа, т.к. эти значения понадобятся для ответа на поставленные вопросы, для этого воспользуемся методом **'dropna()'**
* затем уберем минус, округлим числа и поменяем на целочисленный тип методами **astype(), replace(), round()**(для большей точности)

In [25]:
data.dropna(subset = ['days_employed', 'total_income'], inplace=True)
data = data.reset_index(drop=True)
data['days_employed'] = data['days_employed'].astype(str)
data['days_employed'] = data['days_employed'].replace('-', '', regex=True).astype(float).round(1)
data['days_employed'] = data['days_employed'].astype(int)
#data['days_employed'].head(30)

* в столбце **'days_employed'** прослеживается явная аномалия (стаж в днях у человека не может быть такой величины), скорее всего сбой при выгрузке, не удалось проследить взаимосвязи между аномальными цифрами. Т.к. этот столбец не понадобится нам для ответы на поставленные вопросы оставим так, но **нужно обратить внимание!**.

In [26]:
data['total_income'] = data['total_income'].round(1).astype(int)
#data['total_income'].head(15)

In [27]:
#Заменим неправильные значения в столбце 'children':
data['children'] = data['children'].astype(str)
data['children'] = data['children'].replace('-1', '1', regex=True)
data['children'] = data['children'].replace('20', '2', regex=True)
#data['children'].value_counts()

По суммам значений в столбцах, сразу понятно, что всё получилось :)

Теперь нужно разобраться с полом, т.к. это всего одна ячейка, поставим любой из двух полов, т.к. в составе поставленных задач ни одной связанной с полом нет.

In [28]:
data['gender'] = data['gender'].astype(str)
data['gender'] = data['gender'].replace('XNA', 'F', regex=True)
#data['gender'].value_counts()

### 4.Проведём стемминг столбца 'purpose', чтобы упорядочить названия целей кредита <a id='100104'></a>

In [29]:
purpose = data['purpose'].unique()
russian_stemmer = SnowballStemmer('russian')
for word in purpose:
    print(('Исходное слово - {: <40}| ' 'после стемминга -----').format(word), russian_stemmer.stem(word))

Исходное слово - покупка жилья                           | после стемминга ----- покупка жил
Исходное слово - приобретение автомобиля                 | после стемминга ----- приобретение автомобил
Исходное слово - дополнительное образование              | после стемминга ----- дополнительное образован
Исходное слово - сыграть свадьбу                         | после стемминга ----- сыграть свадьб
Исходное слово - операции с жильем                       | после стемминга ----- операции с жил
Исходное слово - образование                             | после стемминга ----- образован
Исходное слово - на проведение свадьбы                   | после стемминга ----- на проведение свадьб
Исходное слово - покупка жилья для семьи                 | после стемминга ----- покупка жилья для сем
Исходное слово - покупка недвижимости                    | после стемминга ----- покупка недвижим
Исходное слово - покупка коммерческой недвижимости       | после стемминга ----- покупка коммерческой недвижим


* Из указанных фраз можно выделить 5 основ - жил, образован, автомобил, свадьб, недвижим. Жилье и недвижимость по сути одно и то же, поэтому сведем их в колонке 'purpose' к одному слову - **недвижимость**, остальные к cловам: **образование, автомобиль, свадьба**. 
*************
* Найдем список фраз, которые нужно заменить, по каждой основе:

In [30]:
for purp_ned in purpose:
    stemmed_ned = russian_stemmer.stem(purp_ned)
    for word in stemmed_ned.split(' '):
        if word == 'недвижим':
            print(purp_ned)

покупка недвижимости
покупка коммерческой недвижимости
покупка жилой недвижимости
строительство собственной недвижимости
недвижимость
строительство недвижимости
операции с коммерческой недвижимостью
операции со своей недвижимостью
операции с недвижимостью
строительство жилой недвижимости


In [31]:
for purp in purpose:
    stemmed_gil = russian_stemmer.stem(purp)
    for word in stemmed_gil.split(' '):
        if word == 'жил':
            print(purp)

покупка жилья
операции с жильем
жилье
покупка своего жилья
ремонт жилью


In [32]:
def ned_purp(data, wrong):
    data['purpose'] = data['purpose'].replace(wrong, 'недвижимость')
    final = data[data['purpose'] == wrong]['purpose'].count()
    return final
ned_purp(data, 'покупка недвижимости')
ned_purp(data, 'покупка коммерческой недвижимости')
ned_purp(data, 'покупка жилой недвижимости')
ned_purp(data, 'строительство собственной недвижимости')
ned_purp(data, 'строительство недвижимости')
ned_purp(data, 'операции с коммерческой недвижимостью')
ned_purp(data, 'строительство жилой недвижимости')
ned_purp(data, 'операции со своей недвижимостью')
ned_purp(data, 'операции с недвижимостью')
ned_purp(data, 'покупка жилья')
ned_purp(data, 'операции с жильем')
ned_purp(data, 'жилье')
ned_purp(data, 'покупка своего жилья')
ned_purp(data, 'ремонт жилью')

0

In [33]:
for purp in purpose:
    stemmed_svad = russian_stemmer.stem(purp)
    for word in stemmed_svad.split(' '):
        if word == 'свадьб':
            print(purp)

сыграть свадьбу
на проведение свадьбы
свадьба


In [34]:
def svad_purp(data, wrong):
    data['purpose'] = data['purpose'].replace(wrong, 'свадьба')
    final = data[data['purpose'] == wrong]['purpose'].count()
    return final
svad_purp(data, 'сыграть свадьбу')
svad_purp(data, 'на проведение свадьбы')

0

In [35]:
for purp in purpose:
    stemmed_avto = russian_stemmer.stem(purp)
    for word in stemmed_avto.split(' '):
                if word == 'автомобил':
                        print(purp)

приобретение автомобиля
на покупку подержанного автомобиля
на покупку своего автомобиля
сделка с подержанным автомобилем
автомобиль
свой автомобиль
сделка с автомобилем
на покупку автомобиля


In [36]:
def avto_purp(data, wrong):
    data['purpose'] = data['purpose'].replace(wrong, 'автомобиль')
    final = data[data['purpose'] == wrong]['purpose'].count()
    return final
avto_purp(data, 'приобретение автомобиля')
avto_purp(data, 'на покупку подержанного автомобиля')
avto_purp(data, 'на покупку своего автомобиля')
avto_purp(data, 'сделка с подержанным автомобилем')
avto_purp(data, 'свой автомобиль')
avto_purp(data, 'сделка с автомобилем')
avto_purp(data, 'на покупку автомобиля')

0

In [37]:
for purp in purpose:
    stemmed_obr = russian_stemmer.stem(purp)
    for word in stemmed_obr.split(' '):
        if word == 'образован':
            print(purp)

дополнительное образование
образование
заняться образованием
получение образования
получение дополнительного образования
получение высшего образования
профильное образование
высшее образование
заняться высшим образованием


In [38]:
def obr_purp(data, wrong):
    data['purpose'] = data['purpose'].replace(wrong, 'образование')
    final = data[data['purpose'] == wrong]['purpose'].count()
    return final
obr_purp(data, 'дополнительное образование')
obr_purp(data, 'заняться образованием')
obr_purp(data, 'получение образования')
obr_purp(data, 'получение дополнительного образования')
obr_purp(data, 'получение высшего образования')
obr_purp(data, 'профильное образование')
obr_purp(data, 'высшее образование')
obr_purp(data, 'заняться высшим образованием')

0

In [39]:
data['purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба',
       'покупка жилья для семьи', 'автомобили', 'покупка жилья для сдачи'],
      dtype=object)

3 значения (автомобили, покупка жилья для семьи и покупка жилья для сдачи) не попали в списки, распределим по категориям:

In [40]:
avto_purp(data, 'автомобили')
ned_purp(data, 'покупка жилья для семьи')
ned_purp(data, 'покупка жилья для сдачи')

0

Проверим уникальные значения столбца **'purpose'**:

In [41]:
data['purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

* Для ответов на поставленные вопросы нам нужны столбцы **'children', 'family_status', 'total_income', 'debt'** 
* составим отдельную таблицу:

In [42]:
data_grouped = data[['children', 'family_status', 'total_income', 'debt']]
data_grouped.head(15)

,children,family_status,total_income,debt
0,1,женат / замужем,253875,0
1,1,женат / замужем,112080,0
2,0,женат / замужем,145886,0
3,3,женат / замужем,267628,0
4,0,гражданский брак,158616,0
5,0,гражданский брак,255763,0
6,0,женат / замужем,240526,0
7,0,женат / замужем,135823,0
8,2,гражданский брак,95856,0
9,0,женат / замужем,144425,0


### 5.Посмотрим на задолженность у клиентов с разным количеством детей: <a id='100105'></a>

In [43]:
group_child = data_grouped.groupby('children')['debt'].sum().sort_values(ascending=False)
group_child

children
0    952
1    409
2    185
3     22
4      3
5      0
Name: debt, dtype: int64

In [44]:
ratio_debt_child = 952 / (409 + 185 + 22 + 3)
ratio_debt_child

1.5379644588045234

In [45]:
data.groupby('children')['debt'].count().sort_values(ascending=False)

children
0    12710
1     4387
2     1918
3      294
4       34
5        8
Name: debt, dtype: int64

In [46]:
ratio_children = 12710 / (4387 + 1918 + 294 + 34 + 8)
ratio_children

1.91386839331426

* Из этой таблицы уже понятно, что у людей с 5 детьми задолженностей по кредиту нет вообще, а процент бездетных среди должников самый большой. Сравнивать эти две категории, конечно, не совсем корректно(общее соотношение бездетных/детных без долга почти 2 к 1), но и если сложить всех должников без детей, то их всё равно в 1,5 раза больше чем должников с детьми.

### 6.Посмотрим на задолженность клиентов в зависимости от их семейного статуса: <a id='100106'></a>

In [47]:
group_family_stat = data_grouped.groupby('family_status')['debt'].sum().sort_values(ascending=False)
group_family_stat

family_status
женат / замужем          846
гражданский брак         339
не женат / не замужем    254
в разводе                 76
вдовец / вдова            56
Name: debt, dtype: int64

In [48]:
ratio_group_family = (846+339)/(254+76+56)
ratio_group_family

3.069948186528497

In [49]:
data.groupby('family_status')['debt'].count().sort_values(ascending=False)

family_status
женат / замужем          11143
гражданский брак          3735
не женат / не замужем     2525
в разводе                 1083
вдовец / вдова             865
Name: debt, dtype: int64

In [50]:
ratio_family = (11143+3735)/(2525+1083+865)
ratio_family

3.326179298010284

Здесь так же явно видна зависисмость между задолженностью и браком. Люди находящиеся в браке + считающие себя в браке практически в 3 раза чаще создают задолженности по кредиту, чем свободные. При этом общее соотношение практически такое же.

In [51]:
group_income = data_grouped['total_income']

print('MEAN - ', group_income.mean())
print('MEDIAN - ', group_income.median())
print('MAX - ', group_income.max())
print('MIN - ', group_income.min())
print()
print()

# также эти значения можно найти методом **describe()**
group_income = group_income.describe().astype(int)
print(group_income)

MEAN -  167421.8523073743
MEDIAN -  145017.0
MAX -  2265604
MIN -  20667


count      19351
mean      167421
std       102971
min        20667
25%       103053
50%       145017
75%       203435
max      2265604
Name: total_income, dtype: int32


### 7. Сформируем группы по доходам: <a id='100107'></a>

In [52]:
data['total_income'].astype(str)

def c_income(income):
    if income <= 130000:
        return 'низкий'
    elif income <= 250000:
        return 'средний'
    elif income <= 500000:
        return 'высокий'
    elif income <= 1000000:
        return 'очень высокий'
    return 'крайне высокий'

data['c_income'] = data['total_income'].apply(c_income)
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,c_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,низкий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,средний


Посчитаем количество групп:

In [53]:
data['c_income'].value_counts(ascending=False)

средний           8596
низкий            7943
высокий           2590
очень высокий      197
крайне высокий      25
Name: c_income, dtype: int64

Общее количество людей с низким и средним доходом к людям с высокими доходами практически 6 к 1:

In [54]:
ratio_income = (8596+7943)/(2590+197+25)
ratio_income

5.881578947368421

### 8.Проследим зависимости от трех факторов - детей, семейного положения, доходов:  <a id='100108'></a>

In [55]:
def addiction_children_debt(row):
    children = row['children']
    debt = row['debt']
    if children == '0':
        if debt == 1:
            return 'долг если нет детей'
    if children == '1':
        if debt == 1:
            return 'долг если 1 ребенок'
    if children == '2':
        if debt == 1:
            return 'долг если 2 детей'
    if children == '3':
        if debt == 1:
            return 'долг если 3 детей'
    if children == '4':
        if debt == 1:
            return 'долг если 4 детей'
    if children == '5':
        if debt == 1:
            return 'долг если 5 детей'
    return 'НЕТ ДОЛГА'
data['debt_children'] = data.apply(addiction_children_debt, axis=1)
data['debt_children'].value_counts(ascending=False)

НЕТ ДОЛГА              17780
долг если нет детей      952
долг если 1 ребенок      409
долг если 2 детей        185
долг если 3 детей         22
долг если 4 детей          3
Name: debt_children, dtype: int64

In [56]:
def addiction_famstat_debt(row):
    family = row['family_status']
    debt = row['debt']
    if family == 'женат / замужем':
        if debt == 1:
            return 'долг если официальный брак'
    if family == 'гражданский брак':
        if debt == 1:
            return 'долг если гражданский брак'
    if family == 'не женат / не замужем':
        if debt == 1:
            return 'долг если нет семьи'
    if family == 'в разводе':
        if debt == 1:
            return 'долг если в разводе'
    if family == 'вдовец / вдова':
        if debt == 1:
            return 'долг если вдовец / вдова'
    return 'НЕТ ДОЛГА'
data['debt_family_status'] = data.apply(addiction_famstat_debt, axis=1)
data['debt_family_status'].value_counts(ascending=False)

НЕТ ДОЛГА                     17780
долг если официальный брак      846
долг если гражданский брак      339
долг если нет семьи             254
долг если в разводе              76
долг если вдовец / вдова         56
Name: debt_family_status, dtype: int64

In [57]:
def addiction_income_debt(row):
    income = row['c_income']
    debt = row['debt']
    if income == 'крайне высокий':
        if debt == 1:
            return 'долг при крайне высоком доходе'
    if income == 'очень высокий':
        if debt == 1:
            return 'долг при очень высоком доходе'
    if income == 'высокий':
        if debt == 1:
            return 'долг при высоком'
    if income == 'средний':
        if debt == 1:
            return 'долг при среднем доходе'
    if income == 'низкий':
        if debt == 1:
            return 'долг при низком доходе'
    return 'НЕТ ДОЛГА'
data['debt_income'] = data.apply(addiction_income_debt, axis=1)
data['debt_income'].value_counts(ascending=False)

НЕТ ДОЛГА                         17780
долг при среднем доходе             730
долг при низком доходе              647
долг при высоком                    180
долг при очень высоком доходе        12
долг при крайне высоком доходе        2
Name: debt_income, dtype: int64

* Меньше всего долгов по кредиту у категорий с высокими доходами, а больше всего со низкими и средними доходами. Эти категории в таблице имеют соотношение 6 к 1.
**********
* Посмотрим какое соотношение у клиентов с долгами:

In [58]:
ratio_debt_income = (730+647)/(180+12+2)
ratio_debt_income

7.097938144329897

### Выводы: <a id='100109'></a>

В полученных данных были обнаружены *отрицательные значения, пропуски, нулевые значения, различные регистры в ячейках*, а так же явно *неправильные значения*, как полностью случайные так и просто случайные. *Нет единообразия* в заполнении колонки с целями кредита.

**Причиной** этого скорее всего является небрежность заполнения и отстутствие четкой инструкции по заполнению таблицы.

Обращает на себя внимание **неслучайная** аномалия в столбце со стажем работы клиента, возможно это произошло в результате выгрузки или какого-либо программного сбоя, т.к. единого признака не удалось увидеть, иногда это двойные числа, возможно у кого-то кнопки на клавиатуре западают, в результате обедов над ней ;)) 

Нулевые значения в колонках "days_employed"  и "total_income" оказались связаны, строки с созависимыми пропусками пришлось удалить, т.к. это не позволяло сделать адекватный анализ данных, а это 10% от списка! **Нужно обратить внимание!**

Нулевые значения в колонке возраст были заменены на средний возраст, т.к. значение этой колонки не было принципиально важным, но на изначальное заполнение так же **нужно обратить внимание!**

Отрицательные и неправильные значения были изменены напрямую ввиду явной случайности при заполнении.

Для использования методов замены некоторых значений необходимо было изменение типа столбцов, для этого использовался метод **astype()**, т.к. нужны были локальные изменения, а для них этот метод оказался более подходящим.

Так же был проведен стемминг целей кредита, т.к. словосочетаний было немного и низкая "аккуратность" в этом случае не имела значения. Этот метод показался более удобным и быстрым именно в этом случае. 

Было выделено 4 цели кредита: 
* жилье + недвижимость были объединены в "недвижимость", т.к. в данном случае их разделение было не важным для анализа, 
а так же еще 3 категории 
* "автомобиль", "свадьба" и "образование" как цели кредита. 
Короткие слова вполне отражают суть словосочетаний.


* прослеживается зависимость между тем имеет ли клиент детей или нет и тем имеет ли он при этом задолженность по кредиту - клиентов с долгом и без детей больше на 34%, по отношению к тем у кого есть дети. При этом количество детей так же влияет на количество задолженностей! Этот вывод (с небольшим отклонением) подтверждает и соотношение общего количества клиентов с детьми/без детей(35%/65%) и клиентов с задолженностью (с детьми30%/без детей60%);
* (если рассматривать 2 группы: официальный + гражданский брак и нет семьи + в разводе + вдовец/вдова) то прослеживается взаимосвязь - клиентов без семьи с задолженностью на 51% меньше, чем клиентов имеющих семью. Это подтверждает и соотношение общего количества клиентов - с семьей и без семьи (77%/23%), а с задолженностью (с семьей и без (76%/24%)).
* (если рассматривать 2 группы клиентов по доходам: низкий + средний доход и высокий + очень высокий + крайне высокий доходы) то прослеживается взаимосвязь - клиентов с низким и средним доходом с задолженностью на 75% больше, чем клиентов с высоким, очень и крайне высоким доходом. Это подтверждает и соотношение общего количества клиентов - низкий+средний и высокий+очень+крайне высокий (86%/14%), а с задолженностью (низкий+средний и высокий+очень+крайне высокий (88%/12%)).

Из этого анализа можно сделать вывод, что клиент с высоким риском задолженности - это человек с низким или средним доходом состоящий в официальном/неофициальном браке и не имеющий детей.